In [32]:
#https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#training-on-gpu
import torch, torchvision, os, cv2
from torch.utils.data import random_split, Dataset
from torch.nn import Sigmoid, Tanh, Linear, ReLU, Sequential, Conv2d, MaxPool2d, Sigmoid, BatchNorm2d, Flatten, ConvTranspose2d
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

images = np.load('LogImages.npy') #Rade the trianing data.
images = np.moveaxis(images, -1, 1) #Reshape channeL from [B, H, W, C] to [B, C, H, W]
labels = np.load('Labels.npy') #Rade the trianing data. 
labels = labels.reshape(labels.shape[0],1)
# images = images.astype(np.float32)
# labels = labels.astype(np.int)

print(labels.shape)
labels2D = np.zeros((labels.shape[0],2))
for i in range(len(labels)):
    lab = labels[i]
    if lab == 0:
        labels2D[i,0] = 1
    if lab == 1:
        labels2D[i,1] = 1
lengths = [round(len(images)*0.8), round(len(images)*0.2)]
print(lengths)
trainImg, testImg = random_split(images, lengths ,generator=torch.random.manual_seed(42)) #Shuffle data with random seed 42 before split train and test
trainLab, testLab = random_split(labels, lengths ,generator=torch.random.manual_seed(42)) #Shuffle data with random seed 42 before split train and test

print(trainImg[0].shape)
print(trainLab[25])


trainData = [] 
for i in range(len(trainImg)):
    trainData.append([torch.tensor(trainImg[i], dtype=torch.float32), torch.tensor(trainLab[i],dtype=torch.float32)])
trainLoader = torch.utils.data.DataLoader(trainData, shuffle=True, batch_size=5)

testData = []
for i in range(len(testImg)):
    testData.append([torch.tensor(testImg[i], dtype=torch.float32), torch.tensor(testLab[i],dtype=torch.float32)])
testLoader = torch.utils.data.DataLoader(testData, shuffle=False, batch_size=5)

(253, 1)
[202, 51]
(3, 200, 200)
[0]


In [33]:
net = Sequential(
            # Defining 1st 2D convolution layer
            Conv2d(3, 16, kernel_size=3, stride=1, padding=1), #200@3 
            BatchNorm2d(16),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 2nd 2D convolution layer
            Conv2d(16, 8, kernel_size=3, stride=1, padding=1), #100@3 
            BatchNorm2d(8),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 3rd 2D convolution layer
            Conv2d(8, 4, kernel_size=3, stride=1, padding=1), #50@3 
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining 4th 2D convolution layer
            Conv2d(4, 2, kernel_size=3, stride=1, padding=1), #25@3 
            BatchNorm2d(2),
            ReLU(inplace=True),
            Flatten(),
            Linear(2 * 25 * 25, 1),
            Sigmoid()
        )

net = net.cuda()
#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
print(net)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU(inplace=True)
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Conv2d(4, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU(inplace=True)
  

In [34]:
#%%time
n_epoch = 10000
for epoch in range(n_epoch):  # loop over the dataset multiple times
#epoch = 0
#while True:
    train_loss = 0.0
    for i, data in enumerate(trainLoader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].cuda(), data[1].cuda() #Reg
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        labels = torch.reshape(labels, (-1,))
        outputs = torch.reshape(outputs, (-1,))

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss += loss.item()
    logStr = 'Train epoch: %d, Loss: %.10f' % (epoch + 1, train_loss/ (i+1))
    print(logStr)
    epoch += 1

print('Finished Training')

Train epoch: 1, Loss: 0.7140840972
Train epoch: 2, Loss: 0.6712070980
Train epoch: 3, Loss: 0.5529625212
Train epoch: 4, Loss: 0.5044317206
Train epoch: 5, Loss: 0.3999476553
Train epoch: 6, Loss: 0.3005379940
Train epoch: 7, Loss: 0.2226416907
Train epoch: 8, Loss: 0.1664946599
Train epoch: 9, Loss: 0.1190747117
Train epoch: 10, Loss: 0.0954423450
Train epoch: 11, Loss: 0.0593387034
Train epoch: 12, Loss: 0.0368468759
Train epoch: 13, Loss: 0.0254139512
Train epoch: 14, Loss: 0.0268241639
Train epoch: 15, Loss: 0.0346287836
Train epoch: 16, Loss: 0.0211556962
Train epoch: 17, Loss: 0.0138199718
Train epoch: 18, Loss: 0.0118654262
Train epoch: 19, Loss: 0.0098604234
Train epoch: 20, Loss: 0.0083996303
Train epoch: 21, Loss: 0.0084665028
Train epoch: 22, Loss: 0.0072535145
Train epoch: 23, Loss: 0.0064963665
Train epoch: 24, Loss: 0.0057582130
Train epoch: 25, Loss: 0.0058353135
Train epoch: 26, Loss: 0.0051017871
Train epoch: 27, Loss: 0.0045277048
Train epoch: 28, Loss: 0.0046103910
T

Train epoch: 226, Loss: 0.0002589808
Train epoch: 227, Loss: 0.0002678768
Train epoch: 228, Loss: 0.0003037823
Train epoch: 229, Loss: 0.0002605807
Train epoch: 230, Loss: 0.0002494228
Train epoch: 231, Loss: 0.0002752826
Train epoch: 232, Loss: 0.0002604540
Train epoch: 233, Loss: 0.0002681420
Train epoch: 234, Loss: 0.0002581461
Train epoch: 235, Loss: 0.0003696706
Train epoch: 236, Loss: 0.0002728156
Train epoch: 237, Loss: 0.0002704332
Train epoch: 238, Loss: 0.0002521478
Train epoch: 239, Loss: 0.0002528661
Train epoch: 240, Loss: 0.0002578880
Train epoch: 241, Loss: 0.0002563130
Train epoch: 242, Loss: 0.0002595452
Train epoch: 243, Loss: 0.0002360665
Train epoch: 244, Loss: 0.0002507006
Train epoch: 245, Loss: 0.0002790630
Train epoch: 246, Loss: 0.0002322207
Train epoch: 247, Loss: 0.0002400392
Train epoch: 248, Loss: 0.0002153679
Train epoch: 249, Loss: 0.0002196837
Train epoch: 250, Loss: 0.0002377751
Train epoch: 251, Loss: 0.0002139353
Train epoch: 252, Loss: 0.0002343117
T

Train epoch: 448, Loss: 0.0001231188
Train epoch: 449, Loss: 0.0001282672
Train epoch: 450, Loss: 0.0001129795
Train epoch: 451, Loss: 0.0001529176
Train epoch: 452, Loss: 0.0001272011
Train epoch: 453, Loss: 0.0001144771
Train epoch: 454, Loss: 0.0001286507
Train epoch: 455, Loss: 0.0001274781
Train epoch: 456, Loss: 0.0001268268
Train epoch: 457, Loss: 0.0001207216
Train epoch: 458, Loss: 0.0001085759
Train epoch: 459, Loss: 0.0001153816
Train epoch: 460, Loss: 0.0001265048
Train epoch: 461, Loss: 0.0001274960
Train epoch: 462, Loss: 0.0001014798
Train epoch: 463, Loss: 0.0001176451
Train epoch: 464, Loss: 0.0001253360
Train epoch: 465, Loss: 0.0001276081
Train epoch: 466, Loss: 0.0001129032
Train epoch: 467, Loss: 0.0001167315
Train epoch: 468, Loss: 0.0001254284
Train epoch: 469, Loss: 0.0001114703
Train epoch: 470, Loss: 0.0001218271
Train epoch: 471, Loss: 0.0001015589
Train epoch: 472, Loss: 0.0001077847
Train epoch: 473, Loss: 0.0001166852
Train epoch: 474, Loss: 0.0001053965
T

Train epoch: 670, Loss: 0.0000804462
Train epoch: 671, Loss: 0.0000785704
Train epoch: 672, Loss: 0.0000799821
Train epoch: 673, Loss: 0.0000724154
Train epoch: 674, Loss: 0.0000702671
Train epoch: 675, Loss: 0.0000710421
Train epoch: 676, Loss: 0.0000692665
Train epoch: 677, Loss: 0.0000739461
Train epoch: 678, Loss: 0.0000716496
Train epoch: 679, Loss: 0.0000778312
Train epoch: 680, Loss: 0.0000723733
Train epoch: 681, Loss: 0.0000806034
Train epoch: 682, Loss: 0.0000703372
Train epoch: 683, Loss: 0.0000729185
Train epoch: 684, Loss: 0.0000724230
Train epoch: 685, Loss: 0.0000776419
Train epoch: 686, Loss: 0.0000705299
Train epoch: 687, Loss: 0.0000645873
Train epoch: 688, Loss: 0.0000724688
Train epoch: 689, Loss: 0.0000846084
Train epoch: 690, Loss: 0.0001196262
Train epoch: 691, Loss: 0.0000755132
Train epoch: 692, Loss: 0.0000719892
Train epoch: 693, Loss: 0.0000740088
Train epoch: 694, Loss: 0.0000700260
Train epoch: 695, Loss: 0.0000697962
Train epoch: 696, Loss: 0.0000764670
T

Train epoch: 892, Loss: 0.0000568106
Train epoch: 893, Loss: 0.0000570023
Train epoch: 894, Loss: 0.0000531911
Train epoch: 895, Loss: 0.0000480660
Train epoch: 896, Loss: 0.0000532996
Train epoch: 897, Loss: 0.0000531605
Train epoch: 898, Loss: 0.0000521288
Train epoch: 899, Loss: 0.0000555300
Train epoch: 900, Loss: 0.0000546032
Train epoch: 901, Loss: 0.0000485582
Train epoch: 902, Loss: 0.0000506275
Train epoch: 903, Loss: 0.0000477701
Train epoch: 904, Loss: 0.0000471811
Train epoch: 905, Loss: 0.0000531869
Train epoch: 906, Loss: 0.0000488188
Train epoch: 907, Loss: 0.0000537269
Train epoch: 908, Loss: 0.0000588847
Train epoch: 909, Loss: 0.0000551963
Train epoch: 910, Loss: 0.0000600765
Train epoch: 911, Loss: 0.0000488880
Train epoch: 912, Loss: 0.0000623015
Train epoch: 913, Loss: 0.0000575010
Train epoch: 914, Loss: 0.0000460268
Train epoch: 915, Loss: 0.0000512894
Train epoch: 916, Loss: 0.0000613587
Train epoch: 917, Loss: 0.0000537393
Train epoch: 918, Loss: 0.0000521319
T

...

Train epoch: 9105, Loss: 0.0000038151
Train epoch: 9106, Loss: 0.0000033357
Train epoch: 9107, Loss: 0.0000034507
Train epoch: 9108, Loss: 0.0000038362
Train epoch: 9109, Loss: 0.0000035571
Train epoch: 9110, Loss: 0.0000038903
Train epoch: 9111, Loss: 0.0000033036
Train epoch: 9112, Loss: 0.0000036610
Train epoch: 9113, Loss: 0.0000040563
Train epoch: 9114, Loss: 0.0000035453
Train epoch: 9115, Loss: 0.0000037678
Train epoch: 9116, Loss: 0.0000042158
Train epoch: 9117, Loss: 0.0000037452
Train epoch: 9118, Loss: 0.0000040521
Train epoch: 9119, Loss: 0.0000039735
Train epoch: 9120, Loss: 0.0000043131
Train epoch: 9121, Loss: 0.0000032094
Train epoch: 9122, Loss: 0.0000041030
Train epoch: 9123, Loss: 0.0000037957
Train epoch: 9124, Loss: 0.0000034835
Train epoch: 9125, Loss: 0.0000051448
Train epoch: 9126, Loss: 0.0000037935
Train epoch: 9127, Loss: 0.0000036906
Train epoch: 9128, Loss: 0.0000035996
Train epoch: 9129, Loss: 0.0000033764
Train epoch: 9130, Loss: 0.0000047581
Train epoch:

Train epoch: 9321, Loss: 0.0000037170
Train epoch: 9322, Loss: 0.0000041726
Train epoch: 9323, Loss: 0.0000036193
Train epoch: 9324, Loss: 0.0000033040
Train epoch: 9325, Loss: 0.0000051149
Train epoch: 9326, Loss: 0.0000036296
Train epoch: 9327, Loss: 0.0000038508
Train epoch: 9328, Loss: 0.0000035385
Train epoch: 9329, Loss: 0.0000043926
Train epoch: 9330, Loss: 0.0000037063
Train epoch: 9331, Loss: 0.0000038986
Train epoch: 9332, Loss: 0.0000033587
Train epoch: 9333, Loss: 0.0000040843
Train epoch: 9334, Loss: 0.0000039767
Train epoch: 9335, Loss: 0.0000033757
Train epoch: 9336, Loss: 0.0000068369
Train epoch: 9337, Loss: 0.0000041830
Train epoch: 9338, Loss: 0.0000035869
Train epoch: 9339, Loss: 0.0000037513
Train epoch: 9340, Loss: 0.0000040197
Train epoch: 9341, Loss: 0.0000037111
Train epoch: 9342, Loss: 0.0000040396
Train epoch: 9343, Loss: 0.0000041652
Train epoch: 9344, Loss: 0.0000039149
Train epoch: 9345, Loss: 0.0000039012
Train epoch: 9346, Loss: 0.0000038947
Train epoch:

Train epoch: 9537, Loss: 0.0000035622
Train epoch: 9538, Loss: 0.0000035622
Train epoch: 9539, Loss: 0.0000034161
Train epoch: 9540, Loss: 0.0000037380
Train epoch: 9541, Loss: 0.0000033632
Train epoch: 9542, Loss: 0.0000036712
Train epoch: 9543, Loss: 0.0000035415
Train epoch: 9544, Loss: 0.0000042904
Train epoch: 9545, Loss: 0.0000040351
Train epoch: 9546, Loss: 0.0000049990
Train epoch: 9547, Loss: 0.0000033902
Train epoch: 9548, Loss: 0.0000038311
Train epoch: 9549, Loss: 0.0000030582
Train epoch: 9550, Loss: 0.0000037214
Train epoch: 9551, Loss: 0.0000034370
Train epoch: 9552, Loss: 0.0000037579
Train epoch: 9553, Loss: 0.0000037570
Train epoch: 9554, Loss: 0.0000055415
Train epoch: 9555, Loss: 0.0000039815
Train epoch: 9556, Loss: 0.0000036714
Train epoch: 9557, Loss: 0.0000031966
Train epoch: 9558, Loss: 0.0000041274
Train epoch: 9559, Loss: 0.0000049935
Train epoch: 9560, Loss: 0.0000033978
Train epoch: 9561, Loss: 0.0000032850
Train epoch: 9562, Loss: 0.0000032995
Train epoch:

Train epoch: 9753, Loss: 0.0000036089
Train epoch: 9754, Loss: 0.0000032598
Train epoch: 9755, Loss: 0.0000040626
Train epoch: 9756, Loss: 0.0000033257
Train epoch: 9757, Loss: 0.0000037691
Train epoch: 9758, Loss: 0.0000083055
Train epoch: 9759, Loss: 0.0000037509
Train epoch: 9760, Loss: 0.0000033114
Train epoch: 9761, Loss: 0.0000032121
Train epoch: 9762, Loss: 0.0000038245
Train epoch: 9763, Loss: 0.0000034616
Train epoch: 9764, Loss: 0.0000035392
Train epoch: 9765, Loss: 0.0000036991
Train epoch: 9766, Loss: 0.0000036327
Train epoch: 9767, Loss: 0.0000036346
Train epoch: 9768, Loss: 0.0000035225
Train epoch: 9769, Loss: 0.0000035302
Train epoch: 9770, Loss: 0.0000034376
Train epoch: 9771, Loss: 0.0000033287
Train epoch: 9772, Loss: 0.0000031974
Train epoch: 9773, Loss: 0.0000036970
Train epoch: 9774, Loss: 0.0000032405
Train epoch: 9775, Loss: 0.0000037294
Train epoch: 9776, Loss: 0.0000030451
Train epoch: 9777, Loss: 0.0000037042
Train epoch: 9778, Loss: 0.0000035843
Train epoch:

Train epoch: 9969, Loss: 0.0000035781
Train epoch: 9970, Loss: 0.0000031208
Train epoch: 9971, Loss: 0.0000035376
Train epoch: 9972, Loss: 0.0000034175
Train epoch: 9973, Loss: 0.0000034345
Train epoch: 9974, Loss: 0.0000035509
Train epoch: 9975, Loss: 0.0000034210
Train epoch: 9976, Loss: 0.0000032207
Train epoch: 9977, Loss: 0.0000031705
Train epoch: 9978, Loss: 0.0000042368
Train epoch: 9979, Loss: 0.0000036698
Train epoch: 9980, Loss: 0.0000039972
Train epoch: 9981, Loss: 0.0000032719
Train epoch: 9982, Loss: 0.0000032988
Train epoch: 9983, Loss: 0.0000036135
Train epoch: 9984, Loss: 0.0000033668
Train epoch: 9985, Loss: 0.0000030880
Train epoch: 9986, Loss: 0.0000030063
Train epoch: 9987, Loss: 0.0000035546
Train epoch: 9988, Loss: 0.0000037103
Train epoch: 9989, Loss: 0.0000037384
Train epoch: 9990, Loss: 0.0000031944
Train epoch: 9991, Loss: 0.0000030153
Train epoch: 9992, Loss: 0.0000033030
Train epoch: 9993, Loss: 0.0000035481
Train epoch: 9994, Loss: 0.0000033523
Train epoch:

In [46]:
# PATH = './CNN2_SGDlr0.001_BCEL_EP10000_72%.pth'
# torch.save(net.state_dict(), PATH)

In [44]:
correct = 0
total = 0
nb_classes = 2
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for data in trainLoader:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        predicted = torch.round(outputs)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        

print('Accuracy of the network on the %s test images: %d %%' % (total,
    100 * correct / total))

print(confusion_matrix)

Accuracy of the network on the 202 test images: 100 %
tensor([[120.,   0.],
        [  0.,  82.]])


In [48]:
correct = 0
total = 0
nb_classes = 2
confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for data in testLoader:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        predicted = torch.round(outputs)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
        

print('Accuracy of the network on the %s test images: %d %%' % (total,
    100 * correct / total))

print(confusion_matrix)

Accuracy of the network on the 51 test images: 72 %
tensor([[28.,  7.],
        [ 7.,  9.]])
